In [ ]:
import cv2 , os , ipywidgets , traitlets
import numpy as np
import ipywidgets.widgets as widgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera # CSICamera(width=224, height=224, capture_width=640, capture_height=480, capture_filp=2, capture_device=0)
CSICamera.capture_flip = 2

In [ ]:
import os
directory_name = 'dataset'

try:
    os.makedirs(directory_name)
except FileExistsError:
    print('Directories not created becasue they already exist')

In [ ]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
camera = CSICamera(capture_device=0)

In [ ]:
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
face_id = 1
global count
count = 0
number_photo = 100

In [ ]:
def face_recognition(camera_image):
    global count
    face_image = np.copy(camera_image)
    gray = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        face_image = cv2.rectangle(face_image, (x,y), (x+w,y+h), (255,0,0), 2)  
        count += 1
        # Save the captured image into the datasets folder
        if count <= number_photo:
            cv2.imwrite("dataset/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])
            print(count)
    jpeg_image = bgr8_to_jpeg(face_image)
    return jpeg_image

In [ ]:
image_widget = widgets.Image(format='jpeg', width=224, height=224)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=face_recognition)
display(image_widget)

In [ ]:
camera.running = True

def update_image(change):
    image = change['new']
    
camera.observe(update_image, names='value')

In [ ]:
camera.running = False
camera.unobserve(update_image, names='value')